Import packages. In addition to regular repositories, we mostly use `Hyperspy`, `skimage`, `scipy` and the `save_signal`function provided by Magnus Nord to import the mib-file to hspy. `zarr` provides the ability to use zspy-format.

In [ ]:
%matplotlib qt

In [ ]:
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
from converter_nord import save_signal
import pyxem as pxm
from pyxem.utils.io_utils import _parse_hdr
from skimage import data, io, feature
from scipy import ndimage, misc
import zarr
import dask.array as da
import os

Make sure you have Hyperspy 1.7.0 or newer. This is crucial to use the Zspy format.

In [ ]:
!pip list

## <br><br>

## 1 Loading

Load mib-file by using `save_signal` to chunk the data into appropriate sizes that can be loaded in the memory.

### 1.1 From mib to hspy

In [ ]:
data_path = "../../Duncan_Alexander/Merlin detector/22-05-06 PTO_DSO_CL02/Scan4D_02.mib"
# _parse_hdr(data_path)

Dimensions of the data can be seen in Gatan (CTRL+D -> info)

In [ ]:
save_signal(data_path,'4D_converted_test.hspy',256,128,1)

if errorcode: delete hspy file and try again. If you are not allowed to remove the file at this point, simply rename the file in the function above.

In [ ]:
s = hs.load('4D_converted.hspy', lazy=True)

In [ ]:
s.data

### 1.2 Convert to zspy

In [ ]:
s.save('4D_converted.zspy',chunks=(16,16,16,16))

In [ ]:
s = hs.load("4D_converted.zspy", lazy=True)

In [ ]:
s.data

## <br>

### 1.3 Rebinning

Rebin data so it is easier and faster to work with when filtering the data

In [ ]:
s_rebin = s.rebin(scale=(2, 2, 2, 2), rechunk=False)

In [ ]:
s_rebin.compute()

Plot and examine data

In [ ]:
s_rebin.plot()

In [ ]:
s_rebin.T.plot()

## <br>

### 1.4 Convert axes
Not nescessary for analysis. Note that some parameters later on might change due to this convertion, so have that in mind

In [ ]:
s.axes_manager.gui()

Save a diffraction pattern to calculate mrad

In [ ]:
# s.inav[10,10].save('Disk.tif')

In [ ]:
s_ax = s_corr    # Define what data you want to convert the axes of


for i in range(4):
    if i < 2:
        s_ax.axes_manager[i].units = "nm"
        s_ax.axes_manager[i].scale = 0.5028
    else:
        s_ax.axes_manager[i].units = "mrad"
        s_ax.axes_manager[i].scale = 0.032484

s_ax.axes_manager[0].name = "Length"
s_ax.axes_manager[1].name = "Height"

s_ax.axes_manager[2].name = "x"
s_ax.axes_manager[3].name = "y"

s_ax.axes_manager

s_corr = s_ax

## <br><br>

## 2 Correcting for bad pixels

### 2.1 Locate positions of dead pixels in the detector

In [ ]:
s_dead_pixels = s_rebin.find_dead_pixels(dead_pixel_value=0)

### 2.2 Locate positions of cosmic rays

In [ ]:
s_hot_pixels = s_rebin.find_hot_pixels(show_progressbar=True, threshold_multiplier = 10)

### 2.3 Correct for bad pixels (dead and hot)

In [ ]:
s_corr = s_rebin.correct_bad_pixels(s_dead_pixels+s_hot_pixels, show_progressbar=True, inplace=False, lazy_result=True)

In [ ]:
s_corr.plot()

## <br><br>

## 3 Apply Canny filter

### 3.1 Optimize Canny parameters

Check for optimal Canny parameters for the given dataset. `sigma` defines Gaussian blur applied to the disk before edge detection. Try to keep it as low as possible to not blur out the edges too much. It is easier to fill in the circles afterwards than filtering out diffraction contrast. `low_threshold` has little to zero contribution, `high_threshold` usually works best around 250-400.

In [ ]:
x,y = 11,44   # Choose coordinates that are in a challenging region of your dataset


fig,ax = plt.subplots(ncols=3,nrows=3,figsize=(6,6))


    # Vary sigma
    
edges1 = feature.canny(s_corr.inav[x,y].data,sigma=1, low_threshold=10, high_threshold=250)
edges2 = feature.canny(s_corr.inav[x,y].data,sigma=5, low_threshold=10, high_threshold=250)
edges3 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=10, high_threshold=250)



    # Vary low threshold
    
edges4 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=0, high_threshold=250)
edges5 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=10, high_threshold=250)
edges6 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=200, high_threshold=250)



    # Vary high threshold
    
edges7 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=10, high_threshold=50)
edges8 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=10, high_threshold=250)
edges9 = feature.canny(s_corr.inav[x,y].data,sigma=10, low_threshold=10, high_threshold=400)



    # Add subfigures to 3x3 grid

ax[0,0].imshow(edges1,cmap='gray')
ax[0,1].imshow(edges2,cmap='gray')
ax[0,2].imshow(edges3,cmap='gray')

ax[1,0].imshow(edges4,cmap='gray')
ax[1,1].imshow(edges5,cmap='gray')
ax[1,2].imshow(edges6,cmap='gray')

ax[2,0].imshow(edges7,cmap='gray')
ax[2,1].imshow(edges8,cmap='gray')
ax[2,2].imshow(edges9,cmap='gray')


    # Remove tick marks
    
for i in range(3):
    for j in range(3):
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])


        
# Annotate for each row
        
ax[0,0].set_ylabel('$I_{low}$=10, $I_{high}$=250')    
ax[1,0].set_ylabel('$\sigma$=10, $I_{high}$=250')    
ax[2,0].set_ylabel('$\sigma$=10, $I_{low}$=10')    


# Annotate above each subfigure

ax[0,0].set_title('$\sigma$=1')   
ax[0,1].set_title('$\sigma$=5') 
ax[0,2].set_title('$\sigma$=10')  

ax[1,0].set_title('$I_{low}$=0') 
ax[1,1].set_title('$I_{low}$=10') 
ax[1,2].set_title('$I_{low}$=100') 

ax[2,0].set_title('$I_{high}$=10')  
ax[2,1].set_title('$I_{high}$=250')  
ax[2,2].set_title('$I_{high}$=400')  



# plt.axis('off')
fig.tight_layout()
plt.show()    


plt.savefig('canny_parameters.jpg')

### 3.2 Apply Canny filter

Apply the Canny filter to your data with the optimal parameters obtained above

In [ ]:
s_edge = s_corr.map(feature.canny, high_threshold = 1500, low_threshold=200, sigma = 4, show_progressbar = True, inplace = False, lazy_output=True)

Do a check of the Canny filter performance for random places in the dataset

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(6,6))


edges1 = s_edge.inav[10,10]
edges2 = s_edge.inav[5,20]
edges3 = s_edge.inav[30,5]
edges4 = s_edge.inav[50,50]


ax[0,0].imshow(edges1,cmap='viridis')
ax[0,1].imshow(edges2,cmap='viridis')
ax[1,0].imshow(edges3,cmap='viridis')
ax[1,1].imshow(edges4,cmap='viridis')


for i in range(2):
    for j in range(2):
        ax[i,j].axis('off')

fig.tight_layout()
plt.show()  

plt.savefig('canny.jpg')

Examine the Canny-filtered 4D dataset.

In [ ]:
s_edge.plot()

### 3.3 Fill in the Canny-filtered disks to avoid diffraction contrast effects.

In [ ]:
s_fill = s_edge.map(ndimage.binary_fill_holes, inplace = False, lazy_output=True)

In [ ]:
s_fill.plot()

## <br><br>

## 4 Center of mass

### 4.1 COM for x- and y-axis

Perform COM of either the edge detected dataset `s_edge` or the filled in dataset `s_fill`.

In [ ]:
s_com = s_fill.center_of_mass()

In [ ]:
s_com.plot(cmap='viridis')    # If 0 and 1 axis corresponds to x- or y-axis will vary depending on sample orientation.

OBS: To actually save the correct plots you have to close everything else.

In [ ]:
plt.imshow(s_com.inav[0].data)
plt.show()
plt.savefig('COM_0.jpg',bbox_inches='tight')

In [ ]:
plt.imshow(s_com.inav[1].data)
plt.show()
plt.savefig('COM_1.jpg',bbox_inches='tight')

### 4.2 We can add the x- and y-axis for a combined map.

In [ ]:
COM_xy = s_com.inav[0].data+s_com.inav[1].data

In [ ]:
plt.imshow(COM_xy)
plt.savefig('COM_xy.jpg')

## <br><br><br><br><br><br><br><br>

## 5 Useful commands

#### Plot transposed data

In [ ]:
s_corr.T.plot(cmap='viridis')

#### Crop 4D data

In [ ]:
s_new = s.inav[30:70,30:70]

#### Image of diffraction disk (signal dimension)

In [ ]:
s.inav[10,10].plot()
# s.inav[10,10].save('Disk.png')

#### Image of sample (navigator dimension)

In [ ]:
s.isig[10,10].plot()
# s.isig[10,10].save('Sample.png')

#### Access axes manager

In [ ]:
s.axes_manager.gui()

#### Access data information 

In [ ]:
s.data

#### Convert to lazy signal
OBS: changes the signal class 

In [ ]:
s = hs.signals.Signal2D(s_edge).as_lazy()

#### Check chunksize

In [ ]:
s.data.chunksize

#### Center of mass

In [ ]:
s_com = s_edge.center_of_mass(threshold=2, show_progressbar=True)
s_com.plot()

#### Canny filter on a single frame

In [ ]:
edge = feature.canny(s_corr.inav[50,50].data, sigma = 4, high_threshold = 70, low_threshold = 10)

#### Plotting figures

Plot and save a range of diffraction patterns across a line profile across e.g. an a-domain. Can be used for both `s_corr`, `s_edge` and `s_fill` of course. I used the magic tool in Gimp to only select the edge and overlap the edges in Figure 16 in the report.

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

for dx in range(20):
    x = 150 + dx
    y = 16
    ax.imshow(s_corr.inav[x,y],cmap='viridis')
    plt.tight_layout()
    plt.axis('off')
    plt.savefig('a_cross_'+str(x)+'_'+str(y)+'.png')

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(6,6))



edges1 = s_edge.inav[70,20]
edges2 = s_edge.inav[85,20]
edges3 = s_edge.inav[130,20]
edges4 = s_edge.inav[165,20]


ax[0,0].imshow(edges1,cmap='viridis')
ax[0,1].imshow(edges2,cmap='viridis')
ax[1,0].imshow(edges3,cmap='viridis')
ax[1,1].imshow(edges4,cmap='viridis')


for i in range(2):
    for j in range(2):
        ax[i,j].axis('off')

fig.tight_layout()
plt.show()    

In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=3,figsize=(6,6))

edges1 = feature.canny(s_corr.inav[50,50].data,sigma=1, low_threshold=10, high_threshold=250)
edges2 = feature.canny(s_corr.inav[50,50].data,sigma=5, low_threshold=10, high_threshold=250)
edges3 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)


edges4 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=0, high_threshold=250)
edges5 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)
edges6 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=200, high_threshold=250)


edges7 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=50)
edges8 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)
edges9 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=500)



# edges1 = s_out.inav[0,1]
# edges2 = s_out.inav[15,19]
# edges3 = s_out.inav[2,20]
# edges4 = s_out.inav[20,40]

# ax[0].imshow(edges1,cmap='gray')
# ax[1].imshow(edges2,cmap='gray')
# ax[2].imshow(edges3,cmap='gray')
# ax[3].imshow(edges4,cmap='gray')

ax[0,0].imshow(edges1,cmap='gray')
ax[0,1].imshow(edges2,cmap='gray')
ax[0,2].imshow(edges3,cmap='gray')

ax[1,0].imshow(edges4,cmap='gray')
ax[1,1].imshow(edges5,cmap='gray')
ax[1,2].imshow(edges6,cmap='gray')

ax[2,0].imshow(edges7,cmap='gray')
ax[2,1].imshow(edges8,cmap='gray')
ax[2,2].imshow(edges9,cmap='gray')

for i in range(3):
    for j in range(3):
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])

ax[0,0].set_ylabel('$I_{low}$=10, $I_{high}$=250')    
ax[1,0].set_ylabel('$\sigma$=10, $I_{high}$=250')    
ax[2,0].set_ylabel('$\sigma$=10, $I_{low}$=10')    

# plt.axis('off')
fig.tight_layout()
plt.show()    
    

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(6,6))
ax[0,0].imshow(s_out.inav[70,20])
ax[0,1].imshow(s_out.inav[85,20])
ax[1,0].imshow(s_out.inav[130,20])
ax[1,1].imshow(s_out.inav[165,20])